<div align="center">

# ***CA2 - Game***

**Parsa KafshduziBukani - 810102501**

</div>


# <span style="color: #3498db;">Minmax Algorithm</span>

In [1]:
import random
import numpy as np
from math import inf
import time
import pygame

pygame 2.6.1 (SDL 2.28.4, Python 3.12.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class PentagoGame:
    def __init__(self, ui=False, print=False, depth=2, pruning = True):
        self.board = np.zeros((6, 6), dtype=int)
        self.current_player = 1
        self.ui = ui
        self.depth = depth
        self.pruning = pruning
        self.nodes_visited = 0
        self.total_nodes = 0
        self.game_over = False
        self.result = None
        self.selected_block = None
        self.move_stage = 0  # 0: place piece, 1: select block, 2: rotate
        self.temp_piece = None
        self.print = print

        if ui:
            pygame.font.init()
            self.screen = pygame.display.set_mode((800, 600))
            pygame.display.set_caption("Pygame Board")
            # self.font = pygame.font.SysFont("Arial", 20)
            self.show_buttons = False
            self.buttons = {
                "rotate_cw": pygame.Rect(650, 200, 100, 50),
                "rotate_ccw": pygame.Rect(650, 300, 100, 50),
            }
            self.setup_controls()
            self.draw_board()

    def setup_controls(self):
        if self.show_buttons:
            pygame.draw.rect(self.screen, (144, 238, 144), self.buttons["rotate_cw"])   # Light Green
            pygame.draw.rect(self.screen, (173, 216, 230), self.buttons["rotate_ccw"])  # Light Blue

            self.screen.draw_text("CLOCKWISE", self.buttons["rotate_cw"].center)
            self.screen.draw_text("COUNTER-CLOCKWISE", self.buttons["rotate_ccw"].center)



    def hide_rotation_buttons(self):
        self.show_buttons = False

    def show_rotation_buttons(self):
        self.show_buttons = True

    def copy_board(self, board):
        return np.copy(board)

    def rotate_block(self, board, block, direction):
        row_start = (block // 2) * 3
        col_start = (block % 2) * 3
        sub = board[row_start : row_start + 3, col_start : col_start + 3]
        rotated = np.rot90(sub, 3 if direction == "cw" else 1)
        board[row_start : row_start + 3, col_start : col_start + 3] = rotated

    def get_possible_moves(self, board, player):
        moves = []
        for i in range(6):
            for j in range(6):
                if board[i][j] == 0:
                    for block in range(4):
                        for dir in ["cw", "ccw"]:
                            moves.append((i, j, block, dir))
        return moves

    def apply_move(self, board, move, player):
        new_board = self.copy_board(board)
        row, col, block, direction = move
        if new_board[row][col] != 0:
            return None
        new_board[row][col] = player
        self.rotate_block(new_board, block, direction)
        return new_board

    def check_winner(self, board):
        for i in range(6):
            for j in range(6):
                if board[i][j] == 0:
                    continue

                # Horizontal
                if j <= 1 and np.all(board[i, j : j + 5] == board[i][j]):
                    return board[i][j]

                # Vertical
                if i <= 1 and np.all(board[i : i + 5, j] == board[i][j]):
                    return board[i][j]

                # Diagonal
                if (
                    i <= 1
                    and j <= 1
                    and all(board[i + k][j + k] == board[i][j] for k in range(5))
                ):
                    return board[i][j]

                # Anti-diagonal
                if (
                    i <= 1
                    and j >= 4
                    and all(board[i + k][j - k] == board[i][j] for k in range(5))
                ):
                    return board[i][j]
        if np.all(board != 0):
            return 0
        return None
    
    def minimax(self, depth, player, board):
        self.nodes_visited += 1
        
        if depth == 0:
            return self.evaluate_board(board)
        
        moves = self.get_possible_moves(board, player)
        if not moves:
            return self.evaluate_board(board)

        if player == -1:                  # Computer
            max_score = -inf
            for move in moves:
                new_board = self.apply_move(board, move, player)
                score = self.minimax(depth - 1, -player, new_board)

                if score > max_score:
                    max_score = score

            return max_score
        else:                 # Human
            min_score = inf
            for move in moves:
                new_board = self.apply_move(board, move, player)
                score = self.minimax(depth - 1, -player, new_board)

                if score < min_score:
                    min_score = score

            return min_score

    def alpha_beta_pruning(self, depth, player, board, alpha=-inf, beta=inf):
        self.nodes_visited += 1
        
        if depth == 0:
            return self.evaluate_board(board)
        
        moves = self.get_possible_moves(board, player)

        if player == -1:
            max_score = -inf
            for move in moves:
                new_board = self.apply_move(board, move, player)
                if new_board is None: 
                    continue

                score = self.alpha_beta_pruning(depth - 1, -player, new_board, alpha, beta)

                max_score = max(score, max_score)
                alpha = max(alpha, score)
                if beta <= alpha:
                    break
    
            return max_score
        else:
            min_score = inf
            for move in moves:
                new_board = self.apply_move(board, move, player)
                if new_board is None: 
                    continue

                score = self.alpha_beta_pruning(depth - 1, -player, new_board, alpha, beta)

                min_score = min(score, min_score)
                beta = min(beta, score)
                if beta <= alpha:
                    break

            return min_score

    def evaluate_board(self, board):
        WEIGHTS = {
            'two': 5,
            'three': 20,
            'four': 100,
            'five': 100000,
            'center': 10,
            'piece_bonus': 2.5,
            'open_bonus': 0.3
        }

        score = 0
        opponent = lambda p: -p

        def count_sequences(segment, player):
            max_consecutive = 0
            current = 0
            total_pieces = 0
            for pos in segment:
                if pos == player:
                    current += 1
                    total_pieces += 1
                    max_consecutive = max(max_consecutive, current)
                else:
                    current = 0
            base_score = 0
            if max_consecutive >= 5:
                base_score = WEIGHTS['five']
            elif max_consecutive == 4:
                base_score = WEIGHTS['four']
            elif max_consecutive == 3:
                base_score = WEIGHTS['three']
            elif max_consecutive == 2:
                base_score = WEIGHTS['two']
            return base_score + total_pieces * WEIGHTS['piece_bonus']

        # check lines
        for i in range(6):
            row = board[i, :]
            col = board[:, i]
            score += count_sequences(row, -1)
            score -= count_sequences(row, 1)
            score += count_sequences(col, -1)
            score -= count_sequences(col, 1)
                
        # check diagonals
        for offset in range(-2, 3):
            diag = np.diagonal(board, offset=offset)
            anti_diag = np.diagonal(np.fliplr(board), offset=offset)

            for j in range(len(diag) - 4):
                segment = diag[j:j+5]
                score += count_sequences(segment, -1)
                score -= count_sequences(segment, 1)

            for j in range(len(anti_diag) - 4):
                segment = anti_diag[j:j+5]
                score += count_sequences(segment, -1)
                score -= count_sequences(segment, 1)

        # check center blocks
        for i, j in [(1,1), (1,4), (4,1), (4,4)]:
            if board[i, j] == -1:
                score += WEIGHTS['center']
            elif board[i, j] == 1:
                score -= WEIGHTS['center']

        # check open ended blocks
        for i in range(6):
            for j in range(2):
                row = board[i, j:j+5]
                col = board[j:j+5, i]
                if j == 0 and j+5 < 6 and board[i, j+5] == 0:
                    score += count_sequences(row, -1) * WEIGHTS['open_bonus']
                    score -= count_sequences(row, 1) * WEIGHTS['open_bonus']
                if j == 1 and j-1 >= 0 and board[i, j-1] == 0:
                    score += count_sequences(row, -1) * WEIGHTS['open_bonus']
                    score -= count_sequences(row, 1) * WEIGHTS['open_bonus']
                if j == 0 and i+5 < 6 and board[i+5, j] == 0:
                    score += count_sequences(col, -1) * WEIGHTS['open_bonus']
                    score -= count_sequences(col, 1) * WEIGHTS['open_bonus']
                if j == 1 and i-1 >= 0 and board[i-1, j] == 0:
                    score += count_sequences(col, -1) * WEIGHTS['open_bonus']
                    score -= count_sequences(col, 1) * WEIGHTS['open_bonus']
        
        return max(min(score, WEIGHTS['five']), -WEIGHTS['five'])
    
    def get_computer_move(self):
        start_time = time.time()
        best_move = None
        best_value = -inf
        alpha = -inf
        beta = inf

        moves = self.get_possible_moves(self.board, -1)
        if not moves:
            return None
        best_move = moves[0]

        for move in moves:
            if self.game_over:
                break
            new_board = self.apply_move(self.board, move, -1)
            if new_board is None:
                continue

            if self.pruning:
                value = self.alpha_beta_pruning(self.depth-1, 1, new_board, alpha=alpha, beta=beta)
            else:
                value = self.minimax(self.depth-1, 1, new_board)

            if value > best_value:
                best_value = value
                best_move = move

            if alpha < best_value:
                alpha = best_value
            if beta <= alpha:
                break 
        
        if self.print == True:
            print(f"Move took {time.time()-start_time:.2f}s, nodes visited: {self.nodes_visited}")
            
        self.total_nodes += self.nodes_visited
        self.nodes_visited = 0
        return best_move

    def draw_text(self, text, center_pos, max_width):
        font_size = 24
        font = pygame.font.Font(None, font_size)
        text_surface = font.render(text, True, (0, 0, 0))

        text_width = text_surface.get_width()
        if text_width > max_width:
            scale_factor = max_width / text_width
            new_font_size = int(font_size * scale_factor)
            font = pygame.font.Font(None, new_font_size)
            text_surface = font.render(text, True, (0, 0, 0))

        text_rect = text_surface.get_rect(center=center_pos)
        self.screen.blit(text_surface, text_rect)

    def draw_board(self):
        self.screen.fill((0, 0, 0))

        for i in range(6):
            for j in range(6):
                x0 = j * 100
                y0 = i * 100

                if self.board[i][j] == 1:
                    pygame.draw.circle(self.screen, (255, 0, 0), (x0 + 50, y0 + 50), 40)
                elif self.board[i][j] == -1:
                    pygame.draw.circle(self.screen, (0, 0, 255), (x0 + 50, y0 + 50), 40)

                pygame.draw.rect(self.screen, (255, 255, 255), (x0, y0, 100, 100), 1)

        for i in [3, 6]:
            pygame.draw.line(self.screen, (255, 255, 255), (0, i * 100), (600, i * 100), 3)  # Horizontal
            pygame.draw.line(self.screen, (255, 255, 255), (i * 100, 0), (i * 100, 600), 3)  # Vertical

        # Show rotation buttons if in move_stage 2
        if self.move_stage == 2:
            self.highlight_selected_block()
            self.show_rotation_buttons()

        if self.show_buttons:
            pygame.draw.rect(self.screen, (144, 238, 144), self.buttons["rotate_cw"])  # Light Green
            pygame.draw.rect(self.screen, (173, 216, 230), self.buttons["rotate_ccw"])  # Light Blue

            self.draw_text(
                "CLOCKWISE",
                self.buttons["rotate_cw"].center,
                self.buttons["rotate_cw"].width,
            )
            self.draw_text(
                "COUNTER-CLOCKWISE",
                self.buttons["rotate_ccw"].center,
                self.buttons["rotate_ccw"].width,
            )

    def click_handler(self, event):
        if self.game_over or self.current_player != 1:
            return

        x, y = event.pos
        if self.move_stage == 0:  # Place piece
            if x > 600:
                return  # clicks on control area
            col = x // 100
            row = y // 100
            if 0 <= row < 6 and 0 <= col < 6 and self.board[row][col] == 0:
                self.temp_piece = (row, col)
                self.board[row][col] = 1
                self.move_stage = 1
                self.draw_board()

        elif self.move_stage == 1:  # Select block
            if x > 600:
                return
            # which block was clicked
            block_x = 0 if x < 300 else 1
            block_y = 0 if y < 300 else 1
            self.selected_block = block_y * 2 + block_x
            self.move_stage = 2
            self.show_rotation_buttons()
            self.highlight_selected_block()

        elif self.move_stage == 2:  # Rotate
            if self.buttons["rotate_cw"].collidepoint(event.pos):
                self.apply_rotation("cw")
            if self.buttons["rotate_ccw"].collidepoint(event.pos):
                self.apply_rotation("ccw")

    def apply_rotation(self, direction):
        self.rotate_block(self.board, self.selected_block, direction)
        self.current_player = -1
        self.move_stage = 0
        self.selected_block = None
        self.temp_piece = None
        self.hide_rotation_buttons()
        self.draw_board()
        pygame.display.flip()
        self.check_game_over()
        pygame.time.delay(1000)
        self.play_computer_move()

    def highlight_selected_block(self):
        colors = [
            (255, 153, 153),
            (153, 255, 153),
            (153, 153, 255),
            (255, 255, 153),
        ]  # RGB colors

        row_start = (self.selected_block // 2) * 3
        col_start = (self.selected_block % 2) * 3

        pygame.draw.rect(
            self.screen,
            colors[self.selected_block],
            (col_start * 100, row_start * 100, 300, 300),
            5,
        )

    def play_computer_move(self):
        move = self.get_computer_move()
        if move and not self.game_over:
            new_board = self.apply_move(self.board, move, -1)
            if new_board is not None:
                self.board = new_board
                self.current_player = 1
                self.draw_board()
                pygame.display.flip()
                self.check_game_over()
            else:
                print("Invalid computer move!")

    def check_game_over(self):
        winner = self.check_winner(self.board)
        if winner is not None:
            self.game_over = True
            self.result = winner
            print("Game over! Result:", winner)
            if self.ui:
                self.show_game_over_message()

    def show_game_over_message(self):
        self.screen.fill((200, 200, 200))
        pygame.draw.rect(self.screen, (255, 255, 255), (100, 200, 500, 200))
        pygame.draw.rect(self.screen, (0, 0, 0), (100, 200, 500, 200), 3)

        result_text = f"Player {self.result} wins!" if self.result != 0 else "Draw!"
        text_surface = self.font_large.render(result_text, True, (255, 0, 0))
        self.screen.blit(text_surface, (250, 250))

        exit_text = self.font_small.render("Click anywhere to exit", True, (0, 0, 0))
        self.screen.blit(exit_text, (230, 350))
        pygame.display.flip()

    def play(self):
        if self.ui:
            running = True
            while running:
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        running = False
                    elif event.type == pygame.MOUSEBUTTONDOWN:
                        self.click_handler(event)
                self.draw_board()
                pygame.display.flip()
            pygame.quit()
            return self.result
        else:
            while not self.game_over:
                self.print_board()
                winner = self.check_winner(self.board)
                if winner is not None:
                    return winner

                if self.current_player == 1:
                    move = random.choice(self.get_possible_moves(self.board, 1))
                else:
                    move = self.get_computer_move()

                self.board = self.apply_move(self.board, move, self.current_player)
                self.current_player *= -1
            return self.result

    def print_board(self):
        if self.print == False:
            return
        print("-" * 25)
        for row in self.board:
            print(" ".join(f"{x:2}" for x in row))
        print("-" * 25)


**Without Alpha Beta Pruning**

In [ ]:
def evaluate_performance(depths=[1, 2, 3], num_games=5, method=True):
    results = []

    for depth in depths:
        total_time = 0
        total_nodes = 0
        wins = 0
        ties = 0
        losses = 0

        for _ in range(num_games):
            game = PentagoGame(ui=False, print=False, depth=depth, pruning = method)
            game.nodes_visited = 0
            start_time = time.time()
            result = game.play()
            total_time += time.time() - start_time
            total_nodes += game.total_nodes

            if result == -1:
                wins += 1
            elif result == 0:
                ties += 1
            else:
                losses += 1

        avg_time = total_time / num_games
        avg_nodes = total_nodes / num_games
        win_rate = wins / num_games
        results.append({
            'depth': depth,
            'avg_time': avg_time,
            'win_rate': win_rate,
            'avg_nodes': avg_nodes,
            'wins': wins,
            'ties': ties,
            'losses': losses
        })

    return results

def print_evaluation_results(results):
    for res in results:
        print(f"Depth: {res['depth']}")
        print(f"  Avg Time: {res['avg_time']:.4f} seconds")
        print(f"  Win Rate: {res['win_rate']:.4f}")
        print(f"  Avg Nodes: {res['avg_nodes']:.2f}")
        print(f"  Wins: {res['wins']}, Ties: {res['ties']}, Losses: {res['losses']}")
        print()

results = evaluate_performance(depths=[1, 2], num_games=4, method=False)
print_evaluation_results(results)

Depth: 1
  Avg Time: 0.1671 seconds
  Win Rate: 1.0000
  Avg Nodes: 1240.00
  Wins: 4, Ties: 0, Losses: 0

Depth: 2
  Avg Time: 75.8050 seconds
  Win Rate: 1.0000
  Avg Nodes: 469714.00
  Wins: 4, Ties: 0, Losses: 0



**With Pruning**

In [ ]:
results = evaluate_performance(depths=[1, 2], num_games=5, method=True)
print_evaluation_results(results)

Depth: 1
  Avg Time: 0.1759 seconds
  Win Rate: 1.0000
  Avg Nodes: 1240.00
  Wins: 5, Ties: 0, Losses: 0

Depth: 2
  Avg Time: 18.1353 seconds
  Win Rate: 1.0000
  Avg Nodes: 75413.20
  Wins: 5, Ties: 0, Losses: 0



# Heuristic Evaluation Function Properties:

####  1. *Count Sequences*
- Detects the **maximum consecutive pieces** for each player in lines, columns, and diagonals.
- Adds a **base score** + bonus per piece.

####  2. *Board Scanning*
- Checks **rows, columns**, and **both diagonals** for patterns.

####  3. *Center Control*
- Adds bonus if pieces are in central strategic positions.

####  4. *Open-ended Blocks*
- If a sequence has **empty space before or after**, it is considered *open-ended* and given **extra weight**.


# **Questions**

## **1. Effect of Algorithm Depth on Win Rate, Runtime, and Nodes Visited**

1. **Win Rate**:
   - Higher depth (1 to 3) typically increases win rate, as the algorithm evaluates more positions and makes better decisions.
   - Lower depths (1 or 2) yield lower win rates due to limited strategic foresight.

2. **Runtime**:
   - Runtime grows exponentially with depth, as more nodes are explored.
   - Depth 1 is fastest, depth 3 is slowest.

3. **Nodes Visited**:
   - Nodes visited increase exponentially with depth, as the algorithm searches more game tree branches.
   - Depth 3 has the most nodes, depth 1 the least.

**Summary**:
- Deeper search → higher win rate, but much longer runtime and more nodes.
- Shallower search → faster, but lower win rate.

---

## **2. Can Node Children Be Ordered to Maximize Pruning?**

Yes, ordering node children can maximize pruning in alpha-beta pruning.
In alpha-beta pruning, exploring the best moves first increases the chance of early cutoffs.

- **Prioritize Promising Moves**: Sort moves based on a heuristic estimate of their value:
  - Place pieces in center positions (e.g., (1,1), (1,4), (4,1), (4,4)) first, as they control quadrants.
  - Favor rotations that align player pieces into rows/columns/diagonals of three or more.
  - Use `evaluate_board` to score each move’s resulting board; sort moves by descending score for the maximizing player (`-1`) and ascending for the minimizing player (`1`).
- **Implementation**: In `get_possible_moves`, generate all moves (piece placement + rotation), evaluate each resulting board with `evaluate_board`, and sort moves before passing to `alpha_beta_pruning`.

**Impact**:
- Reduces nodes visited, speeding up the search while maintaining optimality.

---

## **3. Branching Factor in the Game**

**Branching Factor** is the **number of possible moves** a player can make at a given point in the game.
- At the **start of the game**, most of the board is empty → many possible moves → **high branching factor**.
- As the **game progresses**, more cells are filled → fewer available moves → **branching factor decreases**.  
  
So, the game starts with many options, but becomes more constrained and easier to search as it continues.

---

## **4. Why Alpha-Beta Pruning Speeds Up Without Losing Accuracy**

- **Speed-Up**:
  - Alpha-beta pruning skips branches that won’t affect the final decision.
  - Reduces nodes explored (e.g., from millions to thousands in Pentago), making it faster.

- **No Accuracy Loss**:
  - Only prunes branches guaranteed to be irrelevant (e.g., a move worse than the best found).
  - Ensures the same optimal move is chosen as minimax, preserving the correct game outcome.

In summary, Prunes useless branches, speeds up search, keeps exact minimax result.

---

## **5. Why Minimax Isn’t Optimal for Random Opponents and What Are the Alternatives**

   Minimax assumes the opponent plays optimally, exploring all moves to find the worst-case outcome.    
   Against a random opponent (like in this project), it overestimates threats, wasting time on unlikely scenarios.

- **Alternative Algorithms**:
  - **Monte Carlo Tree Search (MCTS)**:
    - Simulates many random games from each move to estimate win probabilities.
    - Focuses on promising moves, adapting to random opponent behavior.
    - Faster for random play, as it samples likely outcomes instead of exhaustive search.
  - **Expectimax**:
    - Models random opponent moves with expected values (averaging outcomes).
    - Better suits random play but still slower than MCTS for large games.

In summary, Minimax is too cautious for random opponents; MCTS is faster and more effective by simulating random outcomes.


